# Intro
<font size="3">Welcome to the facial expression bot tutorial.</font>

<img style="float: left; padding:16px" src="https://i.ibb.co/yVFSqgk/Example-Image-Bot.png" alt="Example-Image-Bot" border="0">

This tutorial support you to run your data science project as a telegram bot. To demonstrate the process we use this [dataset](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data). If somebody send an image to the bot, the bot will response with the predicted emotion. Therefore we use a simple CNN trained on this dataset.

The underlying data consists of 48x48 pixel grayscale images of faces. The faces have been automatically registered so that the face is more or less centered and occupies about the same amount of space in each image. So these are the requirements of the input image we have to guarantee. The task is to categorize each face based on the emotion shown in the facial expression in to one of seven categories:

| categorie  | emotion  |
|:---:|:---|
| 0  | Angry  |
| 1  |  Disgust |
| 2  |  Fear |
| 3  |  Happy |
| 4  |  Sad |
| 5  |  Surprise |
| 6  | Neutral |

In this tutorial we focus on creating, deploying and using a classification of facial emotions together with a telegram bot. The underlying model based on the simple neural network of this [notebook](https://www.kaggle.com/drcapa/facial-expression-eda-cnn).

Please notice that the photos sent will not be saved or used for application out of the telegram bot. Of course the usage of the bot is free. 


<font size="3"><span style="color: royalblue;">Please vote the notebook up if it helps you. Feel free to leave a comment above the notebook. Thank you. </span></font>

# Table Of Content
We explain the project stepwise:
1. [Create Telegram Bot](#Telegram)
2. [Create Application](#Application)
3. [Deploy App On Heroku](#Heroku)

# Libraries

In [ ]:
import os
import matplotlib.pyplot as plt

# Path

In [ ]:
path = '/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/'
os.listdir(path)

# Create Telegram Bot <a name="Telegram"></a>
## In General
In this section we describe the creation process of a telegram bot.

1. Open the telegram app and enter **@BotFather** in the search tab. Then select this bot.
2. Click **START** below the chat window.
3. To create a new bot click on **/newbot** or write **/newbot** into the message field.
4. Follow the instructions.
5. After choosing name for the bot and the **username** for the bot you will get your token to access the HTTP API.

Now the bot is available. A user can call/use them via **@username**.

To change the profile photo of the bot write **/setuserpic** in the message field. You can also set a bot description by **/setdescription**.

Remark: Please take the follwoing advice of the BotFather seriously: "Keep your token secure and store it safely, it can be used by anyone to control your bot." 

## In My Case
The name of my bot is **FacialExpressionBot**. The name of the bot and the username are equal.

<img src="https://i.ibb.co/VYgF96p/Facial-Expression-Bot.png" alt="Facial-Expression-Bot" border="1">


## Test The Bot
To test the connection to the bot we create two files:
1. token_bot.py: includes the token **XXX** as a string.
2. test_bot.py: send a simple request and get back the status code.

We use here **XXX** for the token because we don't want to share the true token. 

**token_bot.py**

In [ ]:
token = "XXX"

**test_bot.py**

In [ ]:
import requests
#from token_bot import token

url = "https://api.telegram.org/bot/".replace("bot", "bot"+token)

resp = requests.get(url+'getMe')
print("staus code:", resp.status_code)

Here the code is 404 - not found. This is because of the wrong. If everything is fine you will get the status code 200. The error code are described [here](https://core.telegram.org/api/errors).

# Create Application <a name="Application"></a>
In this section we describe the creation process of the application. We recommend this [notebook](https://www.kaggle.com/drcapa/facial-expression-eda-cnn) as a simple tutorial for defining a simple neural network.Further we presume to develop the app on a local machine.
You can download and use the code of my [github repository](https://github.com/DrCapa/FacialExpressionBot). Now we describe some import parts of the code.
## URL
First we define the URLs for the requests:
```python
from token_bot import token

# URLs
url = "https://api.telegram.org/bot/".replace("bot", "bot"+token)
url_file = "https://api.telegram.org/file/bot/".replace("bot", "bot"+token)
```
We assume that the telegram token is stored in the file token_bot.py as described above. The first URL is used to get the message of a user. The second URL is used to get the files in the attachment of a message.
## Load Model
The next step is to with loading the a pretrained model. With respect to the recommended notebook we saved the model in this way
```python
model_json = model.to_json()
with open("models/model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("models/model.h5")
```
For developing and using the CNN we use the keras library.

So you can load the model by
```python
# Path
path_model = 'model/'
# Load Model
json_file = open(path_model+'model.json', 'r')
model_json = json_file.read()
json_file.close()
model = models.model_from_json(model_json)
model.load_weights(path_model+'model.h5')
```

The next step is to compile the model
```python
# Compile Model
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
```
## Functions
We define some functions. These funcions are so called handler. For further informations use [this documentation](https://python-telegram-bot.readthedocs.io/en/stable/telegram.ext.html#handlers).

## Load & Prepare Image
We focus now on the image preprocessing to guarantee the requirements:
* image size: 48 x 48 pixel,
* image chanel: 1.

Additionally there are two requirements with respect to the photo befor sending:
1. Only one face per photo.
2. The face must be in the center of the image.

In [ ]:
import cv2
import numpy as np

image_size = 48

def image_preprocessing(image, image_size):
    """ Image Preprocessing """

    # Load Image
    readFlag=cv2.COLOR_BGR2GRAY
    #image = np.asarray(bytearray(resp.read()), dtype="uint8")
    #image = cv2.imdecode(image, readFlag)
    image_gray = cv2.cvtColor(image, readFlag)
    
    # Crop Image
    mid_row = int(image_gray.shape[0]/2)
    mid_col = int(image_gray.shape[1]/2)
    if image_gray.shape[0]>image_gray.shape[1]:
        image_cropped = image_gray[mid_row-mid_col:mid_row+mid_col,
                                   0:image_gray.shape[1]]
    else:
        image_cropped = image_gray[0:image_gray.shape[0],
                                   mid_col-mid_row:mid_col+mid_row]
    
    # Rescale Image
    image_rescale = cv2.resize(image_cropped,
                               dsize=(image_size, image_size),
                               interpolation=cv2.INTER_AREA)
    return image_rescale 

To test the image_preprocessing function we consider two test images. 

In [ ]:
path_image = '/kaggle/input/facialemotionbot-images/'
os.listdir(path_image)

In [ ]:
def plot_befor_after(image):
    """ Compare original and prepared image """
    
    fig, axs = plt.subplots(1, 2, figsize=(15, 10))
    fig.subplots_adjust(hspace = .1, wspace=.1)
    axs = axs.ravel()
    # Plot Original Image
    axs[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axs[0].set_title('original shape: '+str(image.shape))
    # Image Preprocessing
    image_rescale = image_preprocessing(image, image_size)
    # Plot Prepared Image
    axs[1].imshow(image_rescale, cmap='gray')
    axs[1].set_title('rescaled shape: '+str(image_rescale.shape))
    for i in range(2):
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
    plt.show()

Landscape:

In [ ]:
file_path = path_image+'landscape.png'
image = cv2.imread(file_path)
plot_befor_after(image)

Portrait

In [ ]:
file_path = path_image+'portrait.png'
image = cv2.imread(file_path)
plot_befor_after(image)

# Deploy On Heroku <a name="Heroku"></a>
In this section we describe the deployment process. To deploy and run the app we use [Heroku](http://). For simple tests Heroku is for free. If you also want to deploy your project on Heroku you can use following instructions.
1. Create an account on Heroku.
2. Create a git repository of your project.
You have to create a requirements.txt file like this
```
requests
telegram
numpy
python-telegram-bot
opencv-contrib-python
tensorflow-cpu
keras
urllib3
```
Remark: We use tensorflow-cpu to reduce memory, compare [this article](https://stackoverflow.com/questions/61062303/deploy-python-app-to-heroku-slug-size-too-large). 
3. Create a new app on Heroku. (There is a helpful documentation [here](https://devcenter.heroku.com/categories/python-support).)

The name of my new app on Heroku is **facial-expression-telegram-bot**. <br>

4. ```$ heroku login ```
5. ```$ heroku git:remote -a facial-expression-telegram-bot ```
6. ```$ git push heroku master```

If the upload was successful you will get a message like this:

<img src="https://i.ibb.co/8XFpvsW/deploy-heroku.png" alt="deploy-heroku" border="0"></a><br />

7. Test your app on the Heroku console
Click on **More** on the upper right conor and select **Run Console**.

<img src="https://i.ibb.co/9ng0z9S/run-console.png" alt="run-console" border="0"></a>

Write **python app.py** in the console line. 